<a href="https://colab.research.google.com/github/i-am-akash/Pyspark_Practice/blob/main/pysparkUdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pyspark UDF Quiz :


*   Read the office Data csv
*   Create a new column increment and provide the increment to the employees on following crietria 


>  1.  If the employee is in NY state his increment woruld be 10% of salary and 5 % of bonus
>  2.If the employee is in CA state his increment woruld be 12% of salary and 3 % of bonus
 
      







In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 56.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=84a107e4bbae3023665a857b00a0a309f117537726ac102c82770bc400e23367
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [15]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
spark = SparkSession.builder.appName("Udf").getOrCreate()

In [10]:
df = spark.read.options( inferschema=True, header=True).csv("/content/sample_data/OfficeData.csv")

In [11]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



This will be part of the udf

In [23]:
def increment(state, salary, bonus):
  increment = 0
  if(state == 'NY'):
    increment = (salary*10)/100+(bonus*5)/100
  
  elif(state == "CA"):
    increment = (salary*12)/100+(bonus*3)/10
  print("reaching")
  return increment

Calling this udf so it will caculary

In [34]:
incrementUDF = udf(lambda x,y,z: increment(x,y,z),DoubleType())

In [35]:
incrementedDF = df.withColumn("Increment",incrementUDF(df.state,df.salary,df.bonus))

In [36]:
incrementedDF.show()

+-------------+----------+-----+------+---+-----+---------+
|employee_name|department|state|salary|age|bonus|Increment|
+-------------+----------+-----+------+---+-----+---------+
|        James|     Sales|   NY| 90000| 34|10000|   9500.0|
|      Michael|     Sales|   NY| 86000| 56|20000|   9600.0|
|       Robert|     Sales|   CA| 81000| 30|23000|  16620.0|
|        Maria|   Finance|   CA| 90000| 24|23000|  17700.0|
|        Raman|   Finance|   CA| 99000| 40|24000|  19080.0|
|        Scott|   Finance|   NY| 83000| 36|19000|   9250.0|
|          Jen|   Finance|   NY| 79000| 53|15000|   8650.0|
|         Jeff| Marketing|   CA| 80000| 25|18000|  15000.0|
|        Kumar| Marketing|   NY| 91000| 50|21000|  10150.0|
+-------------+----------+-----+------+---+-----+---------+

